In [1]:
import qualified Data.Map as Map
import qualified Data.Set as Set
import qualified Data.Set.Ordered as OSet
import Control.Monad.Writer
import Control.Monad.Identity
import Control.Monad.State
import Control.Monad.Reader
import Control.Monad.Trans.Maybe
import Control.Applicative
import Data.List
import Data.Tuple
import Control.Monad.Logger
import Data.Text (pack)

In [2]:
{-# LANGUAGE RankNTypes, KindSignatures #-}
replicateMUntil :: forall (m :: * -> *) a. Monad m => (m a -> m Bool) -> m a -> m [a]
replicateMUntil f ma = do
    res <- f ma
    if res then liftM2 (:) ma (replicateMUntil f ma) else return []

In [3]:
customLog :: MonadLogger m => String -> m ()
customLog msg =  monadLoggerLog (Loc{loc_filename="./general.log", loc_module="Graph Problems", 
                loc_package="Graphs", loc_start=(0,0), loc_end=(0,0)}) (pack "log source") 
                    LevelDebug  (toLogStr $ pack msg)

In [ ]:
type Vertex = Int
type Cost = Int
data Edge = Edge { vertex:: Vertex, cost:: Cost } deriving (Eq, Show, Ord)

In [ ]:
newtype Graph = Graph { adjacencyList :: Map.Map Vertex [Edge] } deriving (Eq, Show)

In [ ]:
constructGraph :: [(Vertex, Edge)] -> Graph
constructGraph edges = Graph $ foldr (\e g -> Map.insertWith (++) (fst e) [snd e] g) Map.empty edges

In [ ]:
graph = constructGraph [(0, Edge 1 2), (1, Edge 0 2), (1, Edge 2 2), (2, Edge 1 2), (2, Edge 3 3),
    (3, Edge 2 3), (1, Edge 3 2), (3, Edge 1 2), (0, Edge 3 1), (3, Edge 0 1), (0, Edge 4 1), (4, Edge 0 1)]
    
graph1 = constructGraph [(0, Edge 3 1), (0, Edge 4 1), (1, Edge 2 1), (2, Edge 1 1), (3, Edge 0 1), (3, Edge 6 1),
    (4, Edge 0 1), (5, Edge 3 1), (5, Edge 6 1), (6, Edge 3 1), (6, Edge 5 1)]

<h4>DFS and BFS</h4>

In [ ]:
getNeighbors :: Vertex -> Graph -> [Vertex]
getNeighbors v g = fmap vertex (Map.findWithDefault [] v (adjacencyList g))

filterVisitedNeighbors :: [Vertex] -> Set.Set Vertex -> [Vertex]
filterVisitedNeighbors listVertices set = filter (\x -> not (Set.member x set)) listVertices

In [ ]:
bfs :: Vertex -> Graph -> [[Vertex]]
bfs s g = go [s] [] g (Set.singleton s) [[s]]
    where go :: [Vertex] -> [Vertex] -> Graph -> Set.Set Vertex -> [[Vertex]] -> [[Vertex]]
          go [] [] g visited result = result
          go [] nextLevel g visited result = go nextLevel [] g visited (result ++ [nextLevel])
          go (x:xs) nextLevel g visited result = 
              go xs (nextLevel ++ unvisitedNeighbors x visited) g (Set.union visited (Set.fromList $ unvisitedNeighbors x visited)) result
          unvisitedNeighbors x = filterVisitedNeighbors (getNeighbors x g)

In [ ]:
bfsWithLogging :: Vertex -> Graph -> WriterT [String] Identity [[Vertex]]
bfsWithLogging s g = go [s] [] g (Set.singleton s) [[s]]
    where go :: [Vertex] -> [Vertex] -> Graph -> Set.Set Vertex -> [[Vertex]] -> WriterT [String] Identity [[Vertex]]
          go [] [] g visited result = return result
          go [] nextLevel g visited result = go nextLevel [] g visited (result ++ [nextLevel])
          go (x:xs) nextLevel g visited result = do
              tell ["Visiting :" ++ show x]
              go xs (nextLevel ++ unvisitedNeighbors x visited) g (Set.union visited (Set.fromList $ unvisitedNeighbors x visited)) result
          unvisitedNeighbors x = filterVisitedNeighbors (getNeighbors x g)

In [ ]:
runIdentity . runWriterT $ bfsWithLogging 0 graph

In [ ]:
type Env = (Graph, Set.Set Vertex)
type DFSState = ([Vertex], Set.Set Vertex)
type Stack = [Vertex]

type DFSResult = ReaderT Env (MaybeT (StateT Vertex Identity)) DFSState

In [ ]:
dfsNextVisit :: Stack -> DFSState -> Maybe ([Vertex], DFSState)
dfsNextVisit [] _ = Nothing
dfsNextVisit (x:xs) (g, visited) = if not (Set.member x visited) 
                                     then Just (x, (xs, Set.insert x visited))
                                   else dfsNextVisit xs (g, visited)                            

In [ ]:
dfs :: Vertex -> Graph -> [Vertex]
dfs s g = go [s] g (Set.empty) []
    where go :: Stack -> Graph -> Set.Set Vertex -> [Vertex] -> [Vertex]
          go [] _ _ dfsTrail = dfsTrail
          go (x:xs) g visited dfsTrail = if not (Set.member x visited)
                                           then go ((getNeighbors x g) ++ xs) g (Set.insert x visited) (dfsTrail ++ [x])
                                         else go xs g visited dfsTrail

In [ ]:
type DFSState = ([Vertex], OSet.OSet Vertex)
type Env = Graph
type DFSNextState = ReaderT Env (StateT DFSState Identity) Int

In [ ]:
{-# LANGUAGE FlexibleContexts #-}

dfsGetNext :: DFSNextState
dfsGetNext = do
    graph <- ask
    s <- get
    let candidates = fst s
    let visited = snd s
    let ret = case candidates of
                []     -> return 0 :: DFSNextState
                (x:xs) -> if x `OSet.member` visited 
                            then put (xs, visited) >> (return 1 :: DFSNextState)
                          else put (getNeighbors x graph ++ xs, x OSet.<| visited) >> (return 1 :: DFSNextState)
    ret

In [ ]:
isNextAvailable :: DFSNextState -> ReaderT Env (StateT DFSState Identity) Bool
isNextAvailable nextState = do
    s <- get
    return (fst s /= [])

In [ ]:
dfsStartState :: DFSState
dfsStartState = ([0], OSet.empty)

dfsSteps = replicateMUntil isNextAvailable dfsGetNext

runStateT (runReaderT dfsSteps graph) dfsStartState

In [ ]:
runStateT (runReaderT dfsSteps graph1) dfsStartState

<h4>Topological Sort</h4>

In [4]:
-- Let us define a more general graph, which can have any type of Vertex, rather than just enforcing it to be Int
data GeneralEdge a b = GeneralEdge { general_vertex:: a, general_cost:: b } deriving (Eq, Show, Ord)

data GeneralGraph a b = GeneralGraph { adjacencyList :: Map.Map a [GeneralEdge a b], 
                            incomingDegree :: Map.Map a Int } deriving (Eq, Show)

In [5]:
addDirectedEdges (a, e) (m, m') = (Map.insertWith (++) a [e] m, 
                                    Map.insertWith (+) (general_vertex e) 1 (Map.insertWith (+) a 0 m'))

constructGeneralDirectedGraph :: (Ord a, Num b) => [(a, GeneralEdge a b)] -> GeneralGraph a b
constructGeneralDirectedGraph edges = uncurry GeneralGraph (foldr addDirectedEdges (Map.empty, Map.empty) edges)

In [6]:
addUndirectedEdges (a, e) (m, m') = (Map.insertWith (++) (general_vertex e) [GeneralEdge a (general_cost e)] 
                                        (Map.insertWith (++) a [e] m),
                                    Map.insertWith (+) a 1 (Map.insertWith (+) (general_vertex e) 1 m'))

constructGeneralUndirectedGraph :: (Ord a, Num b) => [(a, GeneralEdge a b)] -> GeneralGraph a b
constructGeneralUndirectedGraph edges = uncurry GeneralGraph (foldr addUndirectedEdges (Map.empty, Map.empty) edges)

In [7]:
graph' = constructGeneralDirectedGraph [(0, GeneralEdge 1 1), (0, GeneralEdge 2 1), (0, GeneralEdge 3 1)]

In [8]:
getStartVertex :: GeneralGraph a b -> Maybe a
getStartVertex g = fmap fst (find (\(k, d) -> d == 0) (Map.toList $ incomingDegree g))

In [9]:
type TSortState a = ([a], Map.Map a Int)
type GEnv a b = GeneralGraph a b
type TSortNextState a b = ReaderT (GEnv a b) (WriterT [String] (StateT (TSortState a) Identity)) (Maybe a)

In [10]:
tSortNextState :: (Show a, Ord a) => TSortNextState a b
tSortNextState = do
    graph <- ask
    (candidates, degreesMap) <- get
    case candidates of
        []     -> tell ["No more candidates!"] >> return Nothing
        (x:xs) -> tell ["Choosing candidate : " ++ show x] >>
                  put (xs ++ newVerticesWithZeroIncomingDegree, updatedDegreesMap) >>
                  return (Just x)
                    where (newVerticesWithZeroIncomingDegree, updatedDegreesMap) =
                            foldr (\(GeneralEdge v _) (c, m) -> (if m Map.! v == 1 then c ++ [v] 
                                else c, Map.insertWith (+) v (-1) m)) ([], degreesMap) 
                                    (Map.findWithDefault [] x (adjacencyList graph))

In [11]:
isNextAvailableForTSort :: Ord a => TSortNextState a b 
                    -> ReaderT (GEnv a b) (WriterT [String] (StateT (TSortState a) Identity)) Bool
isNextAvailableForTSort nextState = do
    s <- get
    return (fst s /= [])

In [12]:
topologicalSort graph = fmap (runStateT (runWriterT (runReaderT tSortSteps graph))) tSortStartState
    where tSortSteps = replicateMUntil isNextAvailableForTSort tSortNextState
          tSortStartState = fmap (\x -> ([x], incomingDegree graph)) (getStartVertex graph)

In [13]:
topologicalSort graph'

Just (Identity (([Just 0,Just 3,Just 2,Just 1],["Choosing candidate : 0","Choosing candidate : 3","Choosing candidate : 2","Choosing candidate : 1"]),([],fromList [(0,0),(1,0),(2,0),(3,0)])))

In [14]:
graph'' = constructGeneralDirectedGraph [(0, GeneralEdge 1 1), (0, GeneralEdge 2 1), (1, GeneralEdge 2 1),
        (2, GeneralEdge 3 1), (3, GeneralEdge 4 1), (3, GeneralEdge 5 1), (4, GeneralEdge 5 1)]

In [15]:
topologicalSort graph''

Just (Identity (([Just 0,Just 1,Just 2,Just 3,Just 4,Just 5],["Choosing candidate : 0","Choosing candidate : 1","Choosing candidate : 2","Choosing candidate : 3","Choosing candidate : 4","Choosing candidate : 5"]),([],fromList [(0,0),(1,0),(2,0),(3,0),(4,0),(5,0)])))

<h4>Kruskal's Algorithm</h4>

In [17]:
import Data.Array.IArray
import Data.Ix
import Data.Tuple
import Control.Monad.Error
import qualified Data.OrdPSQ as OrdPSQ
import Data.Maybe
import Data.Either
import qualified Data.Set as Set

In [51]:
fst3 :: (a,b,c) -> a
fst3 (x,_,_) = x

snd3 :: (a, b, c) -> b
snd3 (_, y, _) = y

In [ ]:
data UnionFind a = UnionFind { unionSetName :: Array a a, unionSetSize :: Array a Int } deriving Show

In [ ]:
initializeUnionFind :: (Enum a, Ix a) => (a, a) -> UnionFind a
initializeUnionFind (lower, upper) = UnionFind (array (lower, upper) (fmap (\x -> (x, x)) [lower..upper]))
    (array (lower, upper) (fmap (\x -> (x, 1)) [lower..upper]))

In [ ]:
unionOfUnionFinds :: (Enum a, Ix a) => UnionFind a -> a -> a -> UnionFind a
unionOfUnionFinds uf c1 c2
    | sizeArr ! c1 > sizeArr ! c2 = UnionFind (nameArr // [(c2, nameArr ! c1)]) 
        (sizeArr // [(c1, (sizeArr ! c1) + (sizeArr ! c2))])
    | otherwise                   = UnionFind (nameArr // [(c1, nameArr ! c2)]) 
        (sizeArr // [(c2, (sizeArr ! c1) + (sizeArr ! c2))])
    where nameArr = unionSetName uf
          sizeArr = unionSetSize uf
          (lowerBound, upperBound) = bounds nameArr

In [ ]:
data UnionFindResult a = UnionFindResult { parent :: a, updatedUnionFind :: UnionFind a }

findInUnionFind :: (Enum a, Ix a) => UnionFind a -> a -> UnionFindResult a
findInUnionFind uf c = UnionFindResult parent (updateUnionFindWithParent uf parent c)
    where parent = findParentComponent uf c
          (lowerBound, upperBound) = bounds $ unionSetName uf
          findParentComponent uf c
              | c == unionSetName uf ! c = c
              | otherwise                = findParentComponent uf (unionSetName uf ! c)
          updateUnionFindWithParent uf p c
              | p == unionSetName uf ! c = uf
              | otherwise                = updateUnionFindWithParent (UnionFind (unionSetName uf // [(c, p)])
                                                  (unionSetSize uf)) p c

In [ ]:
initializeEdgesPriorityQueue :: (Ord a, Ord b) => GeneralGraph a b -> OrdPSQ.OrdPSQ (a, a) b b
initializeEdgesPriorityQueue graph = Map.foldrWithKey foldEdgeList OrdPSQ.empty (adjacencyList graph)
    where foldEdgeList v edgeList pq = foldr (\e pq -> OrdPSQ.insert (v, general_vertex e) (general_cost e) 
                                            (general_cost e) pq) pq edgeList

In [ ]:
type KruskalMSTState a b = ([(a, a)], OrdPSQ.OrdPSQ (a, a) b b, UnionFind a)
type KruskalEnv a b = GeneralGraph a b
type KruskalMSTNextState a b = ReaderT (KruskalEnv a b) (WriterT [String] (StateT 
                                (KruskalMSTState a b) Identity)) Bool

In [ ]:
unionFindAndUpdate :: (Ix a, Enum a) => (a, a) -> UnionFind a -> (a, a, UnionFind a)
unionFindAndUpdate edge uf = let UnionFindResult c1 uf'  = findInUnionFind uf  (fst edge)
                                 UnionFindResult c2 uf'' = findInUnionFind uf' (snd edge)
                             in (c1, c2, uf'')

In [ ]:
kruskalMSTNextState :: (Ord b, Ix a, Enum a, Ord a, Show a) => KruskalMSTNextState a b
kruskalMSTNextState = do
    graph <- ask
    (edges, pq, uf) <- get
    case OrdPSQ.null pq of
        True -> return False
        False  -> if c1 /= c2
                    then tell ["Choosing edge: " ++ show edge] >> 
                            put (edges ++ [edge], OrdPSQ.deleteMin pq, unionOfUnionFinds uf'' c1 c2) >>
                            return True
                    else tell ["Discarding edge: " ++ show edge] >>
                            put (edges, OrdPSQ.deleteMin pq, uf'') >>
                            return True
                    where
                        (c1, c2, uf'') = unionFindAndUpdate edge uf
                        edge = fst3 (fromJust $ OrdPSQ.findMin pq)

In [ ]:
isNextAvailableForKruskal :: Ord a => KruskalMSTNextState a b -> KruskalMSTNextState a b
isNextAvailableForKruskal nextState = not . OrdPSQ.null . snd3 <$> get

In [ ]:
kruskalMST graph = runStateT (runWriterT (runReaderT kruskalSteps graph)) kruskalStartState
    where kruskalSteps      = replicateMUntil isNextAvailableForKruskal kruskalMSTNextState
          kruskalStartState = ([], initializeEdgesPriorityQueue graph, 
                                  initializeUnionFind (fst lowerBound, fst upperBound))
          orderedKeyList    = Map.toAscList $ adjacencyList graph
          lowerBound        = head orderedKeyList
          upperBound        = last orderedKeyList

In [ ]:
graph''' = constructGeneralDirectedGraph [(0, GeneralEdge 1 2), (0, GeneralEdge 2 4), (0, GeneralEdge 3 8),
                (1, GeneralEdge 0 1), (1, GeneralEdge 2 3), (1, GeneralEdge 3 7), (3, GeneralEdge 2 2)]

kruskalMST graph'''

In [ ]:
graph'''' = constructGeneralUndirectedGraph [(0, GeneralEdge 1 10), (1, GeneralEdge 3 5), (1, GeneralEdge 2 10),
                (2, GeneralEdge 4 7), (3, GeneralEdge 4 5), (4, GeneralEdge 5 15)]
                
kruskalMST graph''''

<h4>Prim's Algorithm</h4>

In [49]:
type PrimMSTState a b = ([(a, a)], OrdPSQ.OrdPSQ (a, a) b b, Set.Set a)
type PrimEnv a b = GeneralGraph a b

In [50]:
type PrimMSTNextState a b = ReaderT (PrimEnv a b) (LoggingT (StateT (PrimMSTState a b) IO)) Bool

In [40]:
checkIfMinIsValid :: (Ord a) => OrdPSQ.OrdPSQ (a, a) b b -> Set.Set a -> Maybe Bool
checkIfMinIsValid pq visited = fmap (not . flip Set.member visited . snd . fst3) (OrdPSQ.findMin pq)

updatePrimPQ :: (Ord a, Ord b) => GeneralGraph a b -> OrdPSQ.OrdPSQ (a, a) b b -> Set.Set a -> a -> OrdPSQ.OrdPSQ (a, a) b b
updatePrimPQ graph pq visited v = foldr (\edge pq' -> if Set.member (general_vertex edge) visited then pq' 
            else OrdPSQ.insert (v, general_vertex edge) (general_cost edge) (general_cost edge) pq') pq 
                (adjacencyList graph Map.! v)
                
fromMaybeBool :: Maybe Bool -> Bool
fromMaybeBool Nothing = False
fromMaybeBool (Just x) = x

In [43]:
primMSTNextState :: (Ord a, Show a, Ord b) => PrimMSTNextState a b
primMSTNextState = do
    graph <- ask
    (edges, pq, visited) <- lift $ lift get
    case OrdPSQ.null pq of
        True -> return False
        False  -> if fromMaybeBool $ checkIfMinIsValid pq visited
                    then lift (customLog $ "Choosing edge: " ++ show edge) >>
                            lift (lift $ put (edges ++ [edge], updatePrimPQ graph (OrdPSQ.deleteMin pq) visited (snd edge), 
                                Set.insert (snd edge) visited)) >>
                            return True
                    else lift (customLog $ "Discarding edge: " ++ show edge) >>
                            lift (lift $ put (edges, OrdPSQ.deleteMin pq, visited)) >>
                            return True
                    where
                        edge = fst3 (fromJust $ OrdPSQ.findMin pq)

In [56]:
isNextAvailableForPrim :: Ord a => PrimMSTNextState a b -> PrimMSTNextState a b
isNextAvailableForPrim nextState = not . OrdPSQ.null . snd3 <$> (lift $ lift get)

Line 2: Move brackets to avoid $
Found:
not . OrdPSQ.null . snd3 <$> (lift $ lift get)
Why not:
not . OrdPSQ.null . snd3 <$> lift (lift get)

In [46]:
-- primMST :: (Show a, Enum a, Ix a, Ord b) => PrimEnv a b -> Identity (([Bool], [String]), PrimMSTState a b)
primMST graph = runStateT (runWriterT (runLoggingT primSteps graph)) primStartStep
    where primSteps         = replicateMUntil isNextAvailableForPrim primMSTNextState
          primStartStep     = ([], updatePrimPQ graph OrdPSQ.empty Set.empty lowerBound, Set.singleton lowerBound)
          orderedKeyList    = Map.toAscList $ adjacencyList graph
          lowerBound        = fst (head orderedKeyList)

: 

In [ ]:
primMST graph''''

<h4>Dijkstra's Algorithm</h4>

In [53]:
:t get

get :: forall s (m :: * -> *). MonadState s m => m s